# Semi-Automated Seagrass Classification Using Earth Engine Python API

This script classify seagrass beds in selected BOA images using ground-data to train the Support Vector Machine classifier. The outputs can be exported to EE Assets. All the training and validation matrices and accuracies can be saved as an Excel file in your working directory.<br/>


Script by: Luis Lizcano-Sandoval (2021)<br/>
College of Marine Sciences, University of South Florida<br/>
luislizcanos@usf.edu
Updated: 05/19/2022

<font size="4">**Workflow:**</font>

1. Import required images, collections, data, etc.
2. Mask clouds and land
3. Apply Depth-Invariant Index (generates band-ratios B2B3)
4. Sample bands: B1, B2, B3, B4, B/G
5. Train models and classify (SVM)
6. Get confusion matrices and accuracies
7. Export classification images to EE Assets (.tiff)
8. Save matrices in GDrive (.xlxs)

In [ ]:
## Run this cell to mount your Google Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

In [ ]:
## Authenticate your EE account
!earthengine authenticate

In [ ]:
## Install some libraries [may need to restart runtime after]:
!pip install xlsxwriter
!pip install geemap

In [ ]:
## Clone github repo:
!git clone https://github.com/luislizcano/seagrass_scripts.git

**Load required libraries:**

In [1]:
## Import google libraries
import os, sys #Import os, sys again if runtime was restarted.
sys.path.insert(0,'/content/seagrass_scripts')
sys.path.append('/content/seagrass_scripts/bin/')

from google.colab import auth
auth.authenticate_user()

import google
SCOPES = ['https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/earthengine']
CREDENTIALS, project_id = google.auth.default(default_scopes=SCOPES)

import ee
ee.Initialize(CREDENTIALS, project='earth-engine-252816')

In [ ]:
## Import other libraries and functions.
## Print EE version and verify if it is loaded:
import pandas as pd
import xlsxwriter
import datetime
import geemap.foliumap as geemap
from functions import CloudScore6S,landMaskFunction,tidalMask,turbidityMask,DII
print('EE version: ',ee.__version__)

**Some metadata:**

In [3]:
## Define the source of your image, if from your EE Assets or EE Collections:
# imageSource = 'assets'  ## Read BOA imagery from EE Assets
imageSource = 'ee'      ## Read BOA imagery from EE Collections

## Define the type of satellite imagery - uncomment as needed:
satellite = 'Sentinel2'
# satellite = 'Landsat8'
# satellite = 'Landsat7'
# satellite = 'Landsat5'

## Settings for running script:
regionName = 'Laguna Madre MX' ## Region to classify [match metadata from ´regions´ collection below]
boaFolder = 'FL_20' ## Name of folder with BOA images in assets.
exportFolder = '2020_pre' ## Name of EE folder to save the final output - must create it manually if not exist yet.
dataFolder = 'Ground-points-19' ## EE Folder with Ground-truth points
smoothStr = '_raw_' # Smooth classified pixels or not? options: '_smooth_' or '_raw_'

## More metadata for saving image properties:
nameCode = 'LMMX' ## Unique code names
regionCountry = 'Mexico'
state = 'Tamaulipas'

**Insert list of image IDs:**

In [4]:
## Paste image ids. The script can work for images from different tiles.
imageID = '20201107T170521_20201107T171231_T14QPM'

**Import data:**

In [ ]:
#########################    IMPORT TRAINING DATA    #########################

# Sandy areas
sand_areas = ee.FeatureCollection("users/lizcanosandoval/ground-points/Sand")
print('Sand areas loaded:', sand_areas.get('system:id').getInfo())

# Ground-Points
groundPoints = ee.FeatureCollection("users/lizcanosandoval/ground-points/"+dataFolder)
#groundPoints = ee.FeatureCollection("users/lizcanosandoval/ground-points/Turkey_2020")
print('Ground-points loaded:', groundPoints.get('system:id').getInfo())

## Landmask
#land = ee.FeatureCollection("users/lizcanosandoval/Florida_10m") ##Created from NDWI using Sentinel-2 imagery
land = ee.ImageCollection("users/lizcanosandoval/Watermask_S2") ##Created from NDWI using Sentinel-2 imagery
print('Land mask loaded:', land.get('system:id').getInfo())

## Region polygons:
regions = ee.FeatureCollection("users/lizcanosandoval/Seagrass/Regions")
print('Regions loaded:', regions.get('system:id').getInfo())

print('*** Collections loaded correctly! ***')

#**Start classification:**

**Prepare image metadata**

In [ ]:
######################   Prepare image metadata  #########################

## If the image source is your asset, then define the folder where the satellite image is:
if 'assets'== imageSource:
    ## Load BOA image from assets:
    if 'Sentinel' in satellite:
        imageTarget = ee.Image("users/lizcanosandoval/BOA/Sentinel/"+boaFolder+'/'+imageID)
    elif 'Landsat' in satellite:
        imageTarget = ee.Image("users/lizcanosandoval/BOA/Landsat/"+boaFolder+'/'+imageID)
    ## Get image metadata:
    imageSat = imageTarget.get('satellite').getInfo() #Image satellite
    imageTile = imageTarget.get('tile_id').getInfo() #Image tile id
    imageDate = imageTarget.get('date').getInfo() #Image date
    imageGeometry = imageTarget.geometry() #Tile geometry.

## If the image source is an EE collection, then define the satellite collection:
if 'ee'== imageSource:
    ## Load BOA image collection from EE cloud:
    if 'Sentinel' in satellite:
        image = ee.Image("COPERNICUS/S2_SR/"+imageID)
        imageTarget = image.divide(10000).set(image.toDictionary(image.propertyNames()))
        imageSat = imageTarget.get('SPACECRAFT_NAME').getInfo() #Image satellite
        imageTile = imageTarget.get('MGRS_TILE').getInfo() #Image tile id
        ee_date = imageTarget.get('GENERATION_TIME').getInfo()
        imageDate = str(datetime.datetime.utcfromtimestamp(ee_date/1000.0)) #Image date
        imageGeometry = imageTarget.geometry() #Tile geometry.
    elif 'Landsat8' == satellite:
        image = ee.Image("LANDSAT/LC08/C01/T1_SR/"+imageID)
        imageTarget = image.divide(10000).set(image.toDictionary(image.propertyNames()))
        imageSat = satellite
        imageTile = str(imageTarget.get('WRS_PATH').getInfo())+str(imageTarget.get('WRS_ROW').getInfo()) #Image tile id
        imageDate = imageTarget.get('SENSING_TIME').getInfo()
        imageGeometry = imageTarget.geometry() #Tile geometry.
    elif 'Landsat7' == satellite:
        image = ee.Image("LANDSAT/LE07/C01/T1_SR/"+imageID)
        imageTarget = image.divide(10000).set(image.toDictionary(image.propertyNames()))
        imageSat = satellite
        imageTile = str(imageTarget.get('WRS_PATH').getInfo())+str(imageTarget.get('WRS_ROW').getInfo()) #Image tile id
        imageDate = imageTarget.get('SENSING_TIME').getInfo()
        imageGeometry = imageTarget.geometry() #Tile geometry.
    elif 'Landsat5' == satellite:
        image = ee.Image("LANDSAT/LT05/C01/T1_SR/"+imageID)
        imageTarget = image.divide(10000).set(image.toDictionary(image.propertyNames()))
        imageSat = satellite
        imageTile = str(imageTarget.get('WRS_PATH').getInfo())+str(imageTarget.get('WRS_ROW').getInfo()) #Image tile id
        imageDate = imageTarget.get('SENSING_TIME').getInfo()
        imageGeometry = imageTarget.geometry() #Tile geometry.

if 'Sentinel' in imageSat:
    imageScale = 10 # Sentinel resolution
else:
    imageScale = 30 # Landsat resolution

## Region of interest:
aoi = regions.filter(ee.Filter.eq('name',regionName))

print('Metadata prepared')

**Map visualization parameters**

In [ ]:
## USING GEEMAP
Map = geemap.Map(center=[40,-100], zoom=4, layer_ctrl=True)
##Map = geemap.Map(ee_initialize=False, layer_ctrl=True, toolbar_ctrl=False)
##Map

## Define the visualization parameters.
if 'Sentinel' or 'Landsat8' in imageSat:
    vizParams = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 0.2,
        'gamma': [1.8, 1.8, 1.8]}
else:
    vizParams = {
        'bands': ['B3', 'B2', 'B1'],
        'min': 0,
        'max': 0.2,
        'gamma': [1.8, 1.8, 1.8]} ##Landsat7/5


## Center the map and display the image.
centroid = imageGeometry.buffer(1).centroid().coordinates().getInfo()
#print(centroid)
lon = centroid[0]
lat = centroid[1]
Map.setCenter(lon, lat, 10)
Map.addLayer(imageTarget, vizParams, 'RGB')
Map

**Mask image**

In [22]:
###########################    CLOUD MASK    #############################

## Recommended Threshold values for
## *Sentinel: 2
## *Landsat: 5
if 'Sentinel' in imageSat:
    threshold = 5
else:
    threshold = 5

## Apply cloud mask
cloudMask = CloudScore6S(imageSat, imageTarget, threshold)


#############################    LAND MASK    ############################

## Apply land mask
#landMask = landMaskFunction(cloudMask, land) ## Use if Land is a featureCollection
landMask = cloudMask.updateMask(land.max()) ## Use if Land is an imageCollection


###################   MASK TIDAL FLATS & TURBIDITY  ######################
## Set parameter values
if 'Sentinel' in imageSat:
    nir = 'B8'
    green = 'B3'
    swir = 'B11'
    blue = 'B2'
elif 'Landsat8' in imageSat:
    nir = 'B5'
    green = 'B3'
    swir = 'B6'
    blue = 'B2'
else:
    nir = 'B4'
    green = 'B2'
    swir = 'B5'
    blue = 'B1'

## Apply tidal flat mask
#ndwiMask = tidalMask(landMask,nir,green)

## Apply turbidity mask for the whole image
#finalMask = turbidityMask(ndwiMask,imageGeometry,nir,swir,blue,land)
finalMask = landMask
print('   Image masked...')

   Image masked...


**Visualize image masked?**

In [ ]:
Map = geemap.Map(center=[40,-100], zoom=4, layer_ctrl=True)
Map.addLayer(finalMask, vizParams, 'Image masked')
Map

**Depth invariant index**

In [ ]:
####################    WATER COLUMN CORRECTION    #######################    

## Filter sand polygons by tile/area:
#sand = ee.FeatureCollection(sand_areas).flatten().filterBounds(imageGeometry)
sand = ee.FeatureCollection(sand_areas).filterBounds(imageGeometry)

## Run the Depth-Invariant Index Function
imageDII = DII(finalMask, imageScale, sand)

print('   Depth-Invariant index applied...')

**Prepare image for classification**

In [ ]:
#########################    SAMPLING BANDS    ###########################
# Classes are:

# 0: Softbottom
# 1: Hardbottom
# 2: Seagrass
# 3: Sparse seagrass //if available

## Filter ground points by AOI and display classes
filterPoints = ee.FeatureCollection(groundPoints).filterBounds(aoi)
print('Ground Points per Class:', filterPoints.aggregate_histogram('class').getInfo())

## Number of points:
totalPoints = filterPoints.size()
print('Total points:', totalPoints.getInfo())

## Select bands to sample. The B/G band is B2B3 in Sentinel-2 and Landsat-8, and B1B2 for Landsat-7/5
if 'Sentinel' in imageSat or 'Landsat8' in imageSat:
    bandsClass = ['B1','B2', 'B3', 'B4','B2B3']
    bg = ['B2B3']
else:
    bandsClass = ['B1','B2', 'B3', 'B1B2']
    bg = ['B1B2']


###################   CLIP TO REGION & APPLY MASKS   #####################
## Apply tidal flat & turbidity masks to specific region of interest:
# seagrass_mask = ee.Image("users/lizcanosandoval/Seagrass/SeagrassMask_FL_100m")
# imageClassify = imageClassify.updateMask(seagrass_mask) #For raster
## Use image with no turbidity mask
## Add B/G band and clip:
imageClassify = landMask.addBands(imageDII.select(bg)).clip(aoi)
## Create masks
mask1 = tidalMask(imageClassify,nir,green)
mask2 = turbidityMask(mask1,aoi,nir,swir,blue,land)

## Add bands of interest to sample training points.
imageClassify = mask2.select(bandsClass)


###########################    APPLY SMOOTHER    #########################
## Define a boxcar or low-pass kernel (Used if want to smooth the image)
smooth = ee.Kernel.euclidean(**{
    'radius': 1, 
    'units': 'pixels', 
    'normalize': True
})

## Apply smoother if set:
if 'smooth' in smoothStr:
    imageClassify = imageClassify.convolve(smooth)

**Visualize pre-classification image?**

In [ ]:
Map = geemap.Map(center=[40,-100], zoom=4, layer_ctrl=True)
Map.addLayer(imageClassify, vizParams, 'Image masked')
Map

**Classify**

In [ ]:
################    GET TRAINING AND VALIDATION DATA    ##################
## Sample multi-spectral data using all ground points.
samplingData = imageClassify.sampleRegions(**{
    'collection': filterPoints,
    'properties': ['class'],
    'scale': imageScale})

## Add random numbers to each feature (from 0 to 1).
randomData = samplingData.randomColumn("random",0)

## Split ground data in training (~70%) and validation (~30%) points
trainingData = randomData.filter(ee.Filter.lt("random",0.7))
validationData = randomData.filter(ee.Filter.gte("random", 0.7))


####################    TRAIN MODELS AND CLASSIFY    #####################
print('   Training models and classifying...')

## Train SVM classifier
SVM = ee.Classifier.libsvm(**{
    'kernelType': 'RBF',
    'gamma': 100,
    'cost': 100
})
trainSVM = SVM.train(**{
    'features': trainingData,
    'classProperty': 'class',
    'inputProperties': bandsClass
})

#### Classify the image using the trained classifier
classifiedSVM = imageClassify.classify(trainSVM)

**Visualize classification output?**

In [ ]:
## Define a palette for the distinct classes
classPalette = ['#3090C7','#CD7F32','#004E00']#,'#78F878']

imgSVM = {
    'min':0,
    'max':2,
    'palette': classPalette
    }

## Visualize classified image:
Map = geemap.Map(center=[lat,lon], zoom=10, layer_ctrl=True)
Map.addLayer(imageTarget, vizParams, 'RGB')
Map.addLayer(imageClassify, vizParams, 'Image masked')
Map.addLayer(classifiedSVM, imgSVM, 'Classified Image')
Map

**Accuracy assessment**

In [ ]:
#######################    TRAINING ACCURACIES    ########################
print('   Getting accuracies...')
## Get a confusion matrix representing resubstitution accuracy.
## {Resubstitution error is the error of a model on the training data.}
## Axis 0 (first level) of the matrix correspond to the input classes (columns), 
## and axis 1 (second level) to the output classes (rows).
matrixTrainingSVM = trainSVM.confusionMatrix()


#######################    VALIDATION ACCURACIES    ######################

## Calculate accuracy using validation data
## Classify the image using the trained classifier
validationSVM = validationData.classify(trainSVM)

## Get a confusion matrix representing expected accuracy (Using validation points - 30%), where:
#  0: Softbottom
#  1: Hardbottom
#  2: Dense Seagrass
#  3: Spare Seagrass

## Axis 0 (the rows) of the matrix correspond to the actual values, 
## and Axis 1 (the columns) to the predicted values.
errorMx = {'actual': 'class', 'predicted': 'classification'}
errorMatrixSVM = validationSVM.errorMatrix(**errorMx)


####################    USER/PRODUCER ACCURACIES    ######################

## Estimate user and producer accuracies
producerAccuracySVM = errorMatrixSVM.producersAccuracy()

# USER
userAccuracySVM = errorMatrixSVM.consumersAccuracy()


#######################    KAPPA COEFFICIENTS    #########################

# The Kappa Coefficient is generated from a statistical test to evaluate the accuracy 
# of a classification. Kappa essentially evaluate how well the classification performed 
# as compared to just randomly assigning values, i.e. did the classification do better 
# than random. The Kappa Coefficient can range from -1 to 1. A value of 0 indicated that 
# the classification is no better than a random classification. A negative number 
# indicates the classification is significantly worse than random. A value close to 1 
# indicates that the classification is significantly better than random.
kappaSVM = errorMatrixSVM.kappa()